In [1]:
from pyspark import SparkContext
sc = SparkContext.getOrCreate()

# MongoDB

- Before, problem with SQL is Impedance Mismatch. But MongoDB supports a variety of data types
- Assume that data will be added or updated, mongoDB will have to calculate and give some "wiggle" room 

## Operations

### Update
- Field Modifier: 
    + `$set`: Create / set the value of a field in a document
    + `$unset`: Removes the specified field from a document
    + `$inc`: Increment the value of the field by a specified amount
    ```
    { $inc: { <field1>: <amount1>, <field2>: <amount2>, ... } }
    ```
    + `$min`/`$max`: Only updates the field if the specified value is less / greater than the existing value
    + `$rename`: Rename the field
    
    ```
    { $rename: { <field1>: <newName1>, <field2>: <newName2>, ... } }
    ```
- Array Modifier: 
    + `$push`: Add an item to the end of an array
    
    ```
    { $push: { <field1>: <value1>, ... } }
    ```
    + `$pop`: Remove the first / last item of an array
    
    ```
    # first element = -1
    { $pop:{ <field> : <-1 or 1>, ...} }
    ```
    + `$pull`: Remove all array element that match a specified query
    
    ```
    { $pull: { <field1>: <value|condition>, <field2>: <value|condition>, ... } }
    ```
    
    + `$`: Acts as a placeholder to update the first element that matches the query condition
        + Positional $ operator identifies an element in an array to update without explicitly specifying the position of the element in the array.
        + It figures out which element of the array the query matched and updates that element  
        
    + `$[]`: Acts as a placeholder to update all elements in an array for the documents that match the query condition
        + To update all elements in an array, see the all positional operator $[]

### Read:
- find(): Return a subetset of documents in a collection
```python
# Key_to_return :1/true-include vs. 0/false-exclude
db.collection_name.find({search_criteria},{key_to_return :1/0})
```
- add criteria for more complex criteria: 

    + `$lt`, `$lte`, `$gt`, `$gte`, `$ne`, `$eq` (less than, less than or equal to, greater than, etc.)
```python
db.collection_name.find( {field:{range_operator:value} } )
```
    + **Or**: `$or` : query values for multiple keys | `$in`, `$nin`: query values for a single key (1 criteria)
    
    ```python
    # or operator
    db.collection_name.find( $or [ {field1: criteria1}, {field2: criteria2} ] )
    
    # in operator
    db.collection_name.find({field: {$in: [value1, value2] }})
    ```
    
    + **Regex**:  Pattern matching strings in queries. Follows Perl Compatible Regular Expression (PCRE) version8.39 with UTF-8 support. 
    ```python
    db.collection_name.find({key:{$regex:pattern})
    ```
    
- query array: 
    + **`$slice`**: Return a subset of elements for an array key, helpful when you know the index of the element.

    ```python
    # positive N = first N elements
    # negative N = last N elements
    db.collection_name.findOne(criteria, {array_name:{$slice: N}})
    ```
    
    + **`$elemMatch`**: Matches documents that contain an array field with **at least** one element that
matches all the specified query criteria. It returns the entire array if at least one matches.
    ```python
    db.collection_name.find({field:{ $elemMatch: {<query1>,<query2>,...}}})
    ```

- query embedded document: 
    + Query for the entire document: Find exact matches of the subdocument.
    + Query for the individual key/value pairs: Using the dot notation to reach into an embedded document.

### Example 1
Load world_bank_project_small.json to ”msan697” database’s “world_bank_project” collection and print only “borrower” information.

```python
# can use true/false
> db.world_bank.find({}, {"borrower": 1, "_id":0})

{ "borrower" : "GOVERNMENT OF ANGOLA" }
{ "borrower" : "GOVERNMENT OF TUNISIA" }
{ "borrower" : "GOVERNMENT OF TANZANIA" }
```


### Example 2 - range

- From “world_bank_project” collection, find the number of documents where their sector1’s Percent is greater than or equal to 60.
- From 1), print only “borrower” and “_id” information.

```python
> db.world_bank.find({}, {"sector1":1, "_id": 0})

{ "sector1" : { "Name" : "Primary education", "Percent" : 100 } }
{ "sector1" : { "Name" : "Public administration- Other social services", "Percent" : 70 } }
{ "sector1" : { "Name" : "Forestry", "Percent" : 50 } }
```

```python
# use "." to go to a deeper level e.g. sector1.Percent
> db.world_bank.find({"sector1.Percent": {$gte:60}}, {"sector1": 1, "_id":0})

{ "sector1" : { "Name" : "Primary education", "Percent" : 100 } }
{ "sector1" : { "Name" : "Public administration- Other social services", "Percent" : 70 } }


> db.world_bank.find({"sector1.Percent": {$gte:60}}, {"borrower": 1, "_id":0})

{ "borrower" : "GOVERNMENT OF ANGOLA" }
{ "borrower" : "GOVERNMENT OF TUNISIA" }
```

### Example 3 - `$or`

Find URLs of document where theme1’s Name is “Water resource management” **or** themecode is 65.

```python
> db.world_bank.find({},{"theme1":1, "themecode":1, "_id":0})

{ "theme1" : { "Name" : "Education for all", "Percent" : 100 }, "themecode" : "65" }
{ "theme1" : { "Name" : "Other economic management", "Percent" : 30 }, "themecode" : "54,24" }
{ "theme1" : { "Name" : "Water resource management", "Percent" : 30 }, "themecode" : "86,82,80,85" }
```

```python
> db.world_bank.find({$or: [{"theme1.Name": "Water resource management"}, {"themecode": "65"} ] },
                    {"theme1": 1, "themecode":1, "url":1, "_id": 0})
                    
{ "theme1" : { "Name" : "Education for all", "Percent" : 100 },
  "themecode" : "65",
  "url" : "http://www.worldbank.org/projects/P122700/angola-learning-all-project?lang=en" }
  
{ "theme1" : { "Name" : "Water resource management", "Percent" : 30 },
  "themecode" : "86,82,80,85",
  "url" : "http://www.worldbank.org/projects/P126361?lang=en" }

```

### Example 4 - `$in`

Find borrowers with impagency is either “MINISTRY OF EDUCATION” or “MINISTRY OF FINANCE”.

```python
> db.world_bank.find({"impagency": {$in: ["MINISTRY OF EDUCATION", "MINISTRY OF FINANCE"]}},
                     {"impagency": 1, "borrower": 1, "_id": 0})
                     
{ "borrower" : "GOVERNMENT OF ANGOLA", "impagency" : "MINISTRY OF EDUCATION" }
{ "borrower" : "GOVERNMENT OF TUNISIA", "impagency" : "MINISTRY OF FINANCE" }
```

### Example 5 - `$regex`
From the world_bank_project collection, find all the project which name is ending with “Project”.

```python
> db.world_bank.find({"project_name": {$regex: "Project$"}}, {"project_name": 1, "_id": 0})
{ "project_name" : "Angola Learning for All Project" }
{ "project_name" : "Kihansi Catchment Conservation and Management Project" }
```

### Example 6 - `$slice`

Find all project_name s ending with “Projects” and return with its project_name and last theme_namecode.

```python
> db.world_bank.find({"project_name": {"$regex": "Project$"}}, {"theme_namecode": 1, "_id":0})

{ "theme_namecode" : [ { "name" : "Education for all", "code" : "65" } ] }
{ "theme_namecode" : [ { "name" : "Water resource management", "code" : "85" },
                       { "name" : "Biodiversity", "code" : "80" },
                       { "name" : "Environmental policies and institutions", "code" : "82" },
                       { "name" : "Other environment and natural resources management", "code" : "86" } ] }
```

```python
> db.world_bank.find({"project_name": {"$regex": "Project$"}}, {"theme_namecode":{$slice:2} , "_id":0})
```

### Example 7 - `$elemMatch`

Find a document which project docs’ DocType is“PID” and project docs’ DocDate is "12-AUG-2013“.
```python
> db.world_bank.find({"projectdocs": {$elemMatch:{"DocType":"PID", "DocDate":"12-AUG-2013"}}}, {"projectdocs":1, "_id": 0})

```

### Example 9

Find a document including
```
{"themecode" : "65",
"totalamt" : 75000000, 
"totalcommamt" : 75000000,
"url" : "http://www.worldbank.org/projects/P122700/angola-learning-all-project?lang=en"}
```

```python
> db.world_bank.find({"themecode" : "65", "totalamt" : 75000000,  "totalcommamt" : 75000000, "url" : "http://www.worldbank.org/projects/P122700/angola-learning-all-project?lang=en"}, {"borrower":1, "themecode":1, "totalamt":1})

{ "_id" : ObjectId("52b213b38594d8a2be17c79e"), "borrower" : "GOVERNMENT OF ANGOLA", "themecode" : "65", "totalamt" : 75000000 }
```

## Aggregation Pipeline

- Returns an array of result documents to the client. (Not write to collections.)
- Nested transformation together: 
    - `db.collecion_name.aggregate({Transform_operator_1 : criteria_1},{Transform_operator_2 : criteria_2}, ...)`
    - `$match, $project, $lookup, $group, $unwind, $out`, etc.

### Pipeline Operators

- `$match` : filters document with criteria
    ```python
    db.world_bank.aggregate({$match:{'totalamt': 75000000}})
    ```

- `$project` : Extract field. Rename the projected field.
    + Specify the inclusion of fields, the addition of new fields, and the resetting the values of existing fields.
    + Mathematical - `$avg`, `$min`, `$max`, `$add`, `$subtract`, `$multiply`, `$pow`, etc.
    
    ```
    # only show field projectdocs
    db.world_bank.aggregate({$match: {"totalamt": 75000000}}, {$project: {"projectdocs":1}} )
    ```
    
- `$unwind` : return each of an array into a separate document.

    ```python
    {"_id": ObjId, "projectdocs": [{A}, {B}] }
    
    # unwind
    {"projectdocs": {A}}
    {"projectdocs": {B}}
    ```
    
    ```
    # example
    # $projectdocs refer to the value of the projectdocs, which is an array
    
    > db.world_bank.aggregate({$match:{'totalamt': 0}},
                            {$project:{"projectdocs":1}},
                            {$unwind: "$projectdocs"} )
    ```

```python
# return 2 elements of projectdocs as 2 separated documents
    { "_id" : ObjectId("52b213b38594d8a2be17c79e"), "projectdocs" : { "DocTypeDesc" : "Project Appraisal Document (PAD),  Vol.1 of 1", "DocType" : "PAD", "EntityID" : "000356161_20130911153455", "DocURL" : "http://www-wds.worldbank.org/servlet/WDSServlet?pcont=details&eid=000356161_20130911153455", "DocDate" : "28-AUG-2013" } }
    
{ "_id" : ObjectId("52b213b38594d8a2be17c79e"), "projectdocs" : { "DocTypeDesc" : "Project Information Document (PID),  Vol.", "DocType" : "PID", "EntityID" : "0000A8056_2012052416140258", "DocURL" : "http://www-wds.worldbank.org/servlet/WDSServlet?pcont=details&eid=0000A8056_2012052416140258", "DocDate" : "24-MAY-2012" } }
```

- `$group` : group documents based on certain fields and combine their values.
    + grouped by the `$group’s _id` field.
    + Accumulator operators can be used together.
    ```
    # assign a group id as the doctype
    > db.world_bank.aggregate({$match:{: 75000000}},
                            {$project:{"projectdocs":1}},
                            {$unwind: "$projectdocs"},
                            {$group: {"_id": "$projectdocs.DocType"}} )
    
    { "_id" : "ISDS"}
    { "_id" : "PID" }
    { "_id" : "PAD" }
    ```
    
    ```
    # count per group doctype
    > db.world_bank.aggregate({$match:{'totalamt': 75000000}},
                            {$project:{"projectdocs":1}},
                            {$unwind: "$projectdocs"},
                            {$group: {"_id": "$projectdocs.DocType",
                                      "count": {$sum:1}}})
    
    { "_id" : "ISDS", "count" : 2 }
    { "_id" : "PID", "count" : 2 }
    { "_id" : "PAD", "count" : 1 }
    ```
    
- `$sort` : collect all document and properly sort them and send the individual shard’s sorted results.
-  `$out` : create/replace a new collection in the current database from the aggregation operation.

### Example 10 
In world_bank_project, 1) find a document where totalcommamt is 75000000 and 2) find the number of projectdocs grouped by DocType belonging to it.

## Distribition Models

### Single Server Distribution Model
- If we can get away without distribution, we should choose a single-server approach. 
- Ex. Graph Database

### Aggregate
- Collection of related objects treated as a unit. 
- Natural unit for distribution.

Two ways for data distribution
1. Sharding : Place different data on different nodes.
2. Replication : Copy the same data over multiple nodes.
    - Master-Slave Replication 
    - Peer-to-Peer Replication

### Sharding
- Distributing data into different servers
- Each of them does its own reads and writes ==> Improves **scalability**.

- Things to consider: 
     - Locate the data commonly accessed together on the same node (Aggregate and/or Data accessed sequentially together.).
     - Physical location.
     - Keep the load even.

- Pros: Improve read & writes
- Cons: Low resilience

### MongoDB & Sharding

- MongoDB supports auto-sharding.
     + Database takes the responsibility of allocating data to shards, balancing data across shards, ensuring data access goes to the right shard.


- **Mongod** : Primary database process that runs on an individual server.
- **Mongos** : Routing process to manage storing different data on different servers and query against the right server.
!

### MongoDB & Replication 

- Master-slave replication : Synchronize slaves with the master.
- Clients can send a master(Primary) read, write, command, index, etc. requests.
- Clients cannot write to slaves (Secondaries).
- By default, clients cannot read from secondaries.
- However by explicitly setting “setSlaveOk()” client can read from secondaries.
- If master dies, you cannot write the data. Mongo can pick one of the saves to become the

#### Read
- Structure
- Master (primary)
    + Authoritative source for the data.
    + Responsible for processing updates. 
    + Manually or automatically assigned.
- Slaves (secondaries)
    + Contains copied data from a master.

#### Write
- Only master can receive the updated written data, which will then propagate to the slave nodes (i.e. the data in the slave node is outdated)

- Always have an odd number of copies for cross-validation of data. Mongo will use the one with the highest voting

#### Pros: 
- Good scalability with intensive read. 
- Read resilience.

#### Cons: 
- Poor with intensive writes. 
- Inconsistency.

### Sharding + Replications 
+ Scalability + Fault Tolerance
+ Master-slave replication and sharding
+ Multiplemasters.
+ Eachdataonlyhasonemaster.
+ A node can be a master for some data and slave for others.

## CAP Theorem

You can only get two: 
- **Consistency**: All nodes have most recent data via eventual consistency. 
- **Availability**: Every request received by a non-failing node must return a response.
- ** Partition Tolerance**:  Clusters can survive from communication breakages in the cluster - - ALWAYS NEED TO BE SATISFIED FOR DIST.COMP.


- ACID addresses an individual node's data consistency. 
- CAP addresses cluster-wide data consis


#### CAP Theorem and Distributed Database
- Requirement: Partition-Tolerance 
- Availability or Consistency??
    + Availability–Shopping
    + Consistency–StockMarket